In [ ]:
import pandas
import os
# reassign the default working dir
os.chdir('C:\\Users\\Cactus\\OneDrive\\Work\\_bestcompany') 

In [ ]:
### Load templates (for future use) ###
import json
with open('Templates.json', 'r', encoding="utf-8") as input_file:
    data = json.loads(input_file.read())

In [62]:
data['Adult']['Large']

'Многопрофильный медицинский центр, коллектив клиники составляет более {Size} врачей различных специальностей. {}Проводится УЗИ сканирование, лабораторная диагностика.'

In [52]:
# os.listdir()

In [233]:
# df = pandas.read_excel('ClinicsRussia.xlsx')
df = pandas.read_excel('ClinicsRussia_excl_green.xlsx',converters={'Phone':str, 'P_size':str})

In [145]:
# number of rows and columns
df.shape

(84, 33)

In [ ]:
### Indexing: label based (=string based) or position based (int based) ###
# dfi.set_index? # ask for help, dir equivalent

In [235]:
# update default index column - NOT in place operator, object needs to be reassigned
dfi = df.set_index('Clinic_ID')
dfi

In [2]:
# dfi = df.set_index('Название клиники')

In [ ]:
# '{}\nРежим работы:\n{}'.format(dfi.loc[238,'Description'], dfi.loc[238,'Working_HRS'])

In [149]:
# dfi.loc[238,'Phone']

In [1]:
# dfi.dtypes

In [239]:
### check data from table ###
id = 239
cdc = {
    'title':dfi.loc[id,'Название клиники'],
    'city':dfi.loc[id,'Город'],
    'working_hrs_open':'9:00',
    'working_hrs_close':'18:00',
    'description':'{desc}\nРежим работы:\n{hrs}'.format(desc=dfi.loc[id,'Description'].format(Size=dfi.loc[id,'P_size']), hrs=dfi.loc[id,'Working_HRS']),
    'phone':dfi.loc[id,'Phone'],
    'address':dfi.loc[id,'Address'],
    'clinic_form':'//*[@id="clinic_form"]'
} # = content dictionary clinic
cdc

{'address': 'ул.Гагарина д.12',
 'city': 'Ярославль',
 'clinic_form': '//*[@id="clinic_form"]',
 'description': 'Крупный многопрофильный стационар на 400 коек.\nРежим работы:\nПн-Пт: с 8:00 до 20:00\nСб: с 8:00 до 18:00 (терапевтическая служба, врачи-специалисты - по графику)\nВс: с 9:00 до 13:00 (терапевтическая служба)',
 'phone': '+74852442747',
 'title': 'Клиническая больница №10',
 'working_hrs_close': '18:00',
 'working_hrs_open': '9:00'}

In [165]:
### Run webdriver ###
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import json
### Login admin page ###
xda = {
    # 'login_btn':'//*[@id="login"]',
    'login_form':'//*[@id="id_username"]',
    'pass_form':'//*[@id="id_password"]',
    'submit_btn':'//*[@id="grp-content-container"]/div/form/div/ul/li/input'
    }
driver = webdriver.Chrome()
# driver.get("https://www.bestcompany.ru/admin/")
driver.get("http://52.174.34.2/admin")

In [166]:
### Log in ### 
driver.find_element_by_xpath(xda['login_form']).send_keys("lapshin@bestcompany.ru")
# driver.find_element_by_xpath(xda['login_form']).send_keys("test@bestcompany.ru")
driver.find_element_by_xpath(xda['pass_form']).send_keys("vfr4mju7")
# driver.find_element_by_xpath(xda['pass_form']).send_keys("testingshit")
driver.find_element_by_xpath(xda['pass_form']).submit()

In [244]:
### Create Xpath dict for clinic page ###
xdc = { 
    'title':'//*[@id="id_title"]',
    'city':'//*[@id="id_city"]',
    'working_hrs_open':'//*[@id="id_from_time"]',
    'working_hrs_close':'//*[@id="id_to_time"]',
    'description':'//*[@id="tinymce"]',
    'phone':'//*[@id="id_phone"]',
    'address':'//*[@id="id_real_address"]',
    'booking_email':'//*[@id="id_booking_email"]'
#     'clinic_form':'//*[@id="clinic_form"]'
    }

In [ ]:
# TEST Extract fields from dataframe and admin page for recording
# cdc = content dictionary clinic
id = 225
cdc = {
    'title':dfi.loc[id,'Название клиники'],
    'city':dfi.loc[id,'Город'],
    'working_hrs_open':'9:00',
    'working_hrs_close':'18:00',
    'description':'{desc}\nРежим работы:\n{hrs}'.format(desc=dfi.loc[id,'Description'].format(Size=dfi.loc[id,'P_size']), hrs=dfi.loc[id,'Working_HRS']),
    'phone':dfi.loc[id,'Phone'],
    'address':dfi.loc[id,'Address'],
    'booking_email':driver.find_element_by_xpath(xdc['booking_email']).text
#     'clinic_form':'//*[@id="clinic_form"]'
    } # = content dictionary clinic
cdc['booking_email'] = '\n'.join([x for x in cdc['booking_email'].splitlines() if x.strip()])

In [3]:
### TEST getting rid of /n/n in the booking_email field
# driver.find_element_by_xpath(xdc['booking_email']).text.splitlines()
# driver.find_element_by_xpath(xdc['booking_email']).text
# 'mikhail@bestcompany.ru\n\nphilipp@bestcompany.ru'.splitlines()
'\n'.join([x for x in 'mikhail@bestcompany.ru\n\nphilipp@bestcompany.ru'.splitlines() if x.strip()])

In [ ]:
### TEST write into db ###
id = 225
driver.get('http://52.174.34.2/admin/booking/clinic/{}/change/'.format(id)) # test server
for k,v in xdc.items(): # iterate over clinic fields
    if k == 'description': # if form is iframe
        driver.switch_to.frame(driver.find_element_by_xpath('//*[@id="id_info_ifr"]'))  ### first switching to iframe
        driver.find_element_by_xpath(xdc[k]).clear() # clear the form
        driver.find_element_by_xpath(xdc[k]).send_keys(cdc[k]) # key from x-path dic is the same as the key from content dic
        driver.switch_to.default_content()  ### switch to default content
    else:
        driver.find_element_by_xpath(xdc[k]).clear()
        driver.find_element_by_xpath(xdc[k]).send_keys(cdc[k])
driver.find_element_by_xpath('//*[@id="clinic_form"]').submit()

In [ ]:
#label based indexing
# df_sadko = dfi.loc["Садко на Академика Сахарова":"Садко на Школьной", "HO_M-F":"HC_WE"]
# df_sadko

In [257]:
# Create ID list 
clinicIDlist = list(df.loc[:, 'Clinic_ID'])
# df.Clinic_ID.tolist() # alternative option

In [258]:
clinicIDlist

[238,
 225,
 229,
 236,
 228,
 235,
 237,
 226,
 233,
 230,
 227,
 234,
 232,
 239,
 266,
 264,
 265,
 269,
 268,
 267,
 245,
 243,
 244,
 248,
 249,
 250]

In [259]:
# Write into a DataBase
for id in clinicIDlist:
    #     driver.get('https://www.bestcompany.ru/admin/booking/clinic/{}/change/'.format(id)) # run main server
    driver.get('http://52.174.34.2/admin/booking/clinic/{}/change/'.format(id)) # run test server
    cdc = {
    'title':dfi.loc[id,'Название клиники'],
    'city':dfi.loc[id,'Город'],
    'working_hrs_open':'9:00',
    'working_hrs_close':'18:00',
    'description':'{desc}\nРежим работы:\n{hrs}'.format(desc=dfi.loc[id,'Description'].format(Size=dfi.loc[id,'P_size']), hrs=dfi.loc[id,'Working_HRS']),
    'phone':dfi.loc[id,'Phone'],
    'address':dfi.loc[id,'Address'],
    'booking_email':driver.find_element_by_xpath(xdc['booking_email']).text
#     'clinic_form':'//*[@id="clinic_form"]'
    } 
    cdc['booking_email'] = '\n'.join([x for x in cdc['booking_email'].splitlines() if x.strip()]) # get rig of excessive line breakes
    for k,v in xdc.items(): # iterate over clinic fields
        if k == 'description': # if form is iframe
            driver.switch_to.frame(driver.find_element_by_xpath('//*[@id="id_info_ifr"]'))  ### first switching to iframe
            driver.find_element_by_xpath(xdc[k]).clear() # clear the form
            driver.find_element_by_xpath(xdc[k]).send_keys(cdc[k]) # key from x-path dic is the same as the key from content dic
            driver.switch_to.default_content()  ### switch to default content
        else:
            driver.find_element_by_xpath(xdc[k]).clear()
            driver.find_element_by_xpath(xdc[k]).send_keys(cdc[k])
    driver.find_element_by_xpath('//*[@id="clinic_form"]').submit()